In [ ]:
import cv2
import easyocr
import numpy as np
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

# OCR engine
reader = easyocr.Reader(['en'], gpu=False)

# Fungsi untuk menghitung posisi tengah Y dari bounding box
def y_center(bbox):
    return np.mean([pt[1] for pt in bbox])

# Fungsi untuk memproses gambar
def proses_gambar():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if not file_path:
        return

    image = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hasil_ocr = reader.readtext(img_rgb)

    if not hasil_ocr:
        hasil_teks.delete('1.0', END)
        hasil_teks.insert(END, "Tidak ada teks terdeteksi.")
        return

    baris = []
    threshold = 25

    for bbox, teks, _ in hasil_ocr:
        y = y_center(bbox)
        masuk = False
        for b in baris:
            if abs(y - b[0][2]) < threshold:
                b.append((bbox, teks, y))
                masuk = True
                break
        if not masuk:
            baris.append([(bbox, teks, y)])

    hasil_atas, hasil_bawah = "", ""
    baris.sort(key=lambda b: min(item[2] for item in b))
    if len(baris) > 0:
        baris[0].sort(key=lambda x: x[0][0][0])
        hasil_atas = " ".join([text for _, text, _ in baris[0]])
    if len(baris) > 1:
        baris[1].sort(key=lambda x: x[0][0][0])
        hasil_bawah = " ".join([text for _, text, _ in baris[1]])

    # Gambar bounding box dan teks
    for bbox, teks, _ in hasil_ocr:
        p1 = tuple(map(int, bbox[0]))
        p2 = tuple(map(int, bbox[2]))
        cv2.rectangle(img_rgb, p1, p2, (0, 255, 0), 2)
        cv2.putText(img_rgb, teks, p1, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Konversi ke PIL dan tampilkan di canvas
    img_tampil = Image.fromarray(img_rgb).resize((600, 400))
    img_tk = ImageTk.PhotoImage(img_tampil)

    canvas.image = img_tk
    canvas.create_image(0, 0, anchor=NW, image=img_tk)

    # Tampilkan hasil teks ke Text widget
    hasil_teks.delete('1.0', END)
    hasil_teks.insert(END, f"Nomor Plat  : {hasil_atas}\n")
    hasil_teks.insert(END, f"Bulan/Tahun : {hasil_bawah}")

# Fungsi untuk menampilkan info aplikasi
def tampilkan_info():
    info_teks = "Abdiel Maulana - 0922040051\nMahaga Sandy   - 0922040057"
    messagebox.showinfo("Info", info_teks)

# ================= GUI SETUP ================= #
root = Tk()
root.title("🔎 Deteksi Plat Nomor Kendaraan - OCR AI")
root.geometry("850x650")
root.configure(bg="#1e1e2e")

# Judul
judul = Label(root, text="🚗 SMART OCR PLAT NOMOR KENDARAAN 🚓", font=("Segoe UI", 16, "bold"), fg="white", bg="#1e1e2e")
judul.pack(pady=15)

# Tombol Unggah
btn_frame = Frame(root, bg="#1e1e2e")
btn_frame.pack()
unggah_btn = Button(btn_frame, text="📁 Unggah & Deteksi Gambar", font=("Segoe UI", 12), fg="white", bg="#4e9eff", activebackground="#256dff", padx=10, pady=5, command=proses_gambar)
unggah_btn.pack(pady=10)

# Canvas untuk menampilkan gambar
canvas = Canvas(root, width=600, height=400, bg="black", highlightthickness=0)
canvas.pack(pady=10)

# Area hasil OCR
hasil_frame = Frame(root, bg="#1e1e2e")
hasil_frame.pack()
Label(hasil_frame, text="📝 Hasil OCR", font=("Segoe UI", 12, "bold"), fg="white", bg="#1e1e2e").pack()
hasil_teks = Text(hasil_frame, height=4, width=60, font=("Consolas", 11), bg="#2e2e3f", fg="#00ffcc")
hasil_teks.pack(pady=5)

# Menu Bar
menu_bar = Menu(root)
info_menu = Menu(menu_bar, tearoff=0)
info_menu.add_command(label="Tentang Kami", command=tampilkan_info)
menu_bar.add_cascade(label="Info", menu=info_menu)
root.config(menu=menu_bar)

# Jalankan aplikasi
root.mainloop()

Using CPU. Note: This module is much faster with a GPU.
